# Graphing module

## Mapping and syntax processing (AST)

### Syntax processing
- Here there are 3 main options available (that I currently know of)
    - ANTLR (more community support)
    - tree-sitter (speed, lazy-evaluation)
    - Bison (more for config files)

Because currently we have the most done on tree-sitter, the focus will be on that, but it's always possible to switch or write a custom AST Parser since our usecase is quite unique

In [28]:
from dataclasses import dataclass
import sys

# This is used to be able to import from src directory
sys.path.append("../")


from src.repository_processing import files_from_repository
from tree_sitter_languages import get_language, get_parser
from tree_sitter import Language, Parser
from pydantic import BaseModel
from programming_language_extensions import currated_language_list

#TODO: Add Result monad to properties to handle errors
# @property is not cached, if more performance needed use @functools.cached_property
@dataclass
class Language:
    """holds language information and parser for AST"""
    name: str

    @property
    def extensions(self) -> list[str]:
        return currated_language_list[self.name]

    @property
    def language(self) -> Language:
        return get_language(self.name)
    
    @property
    def parser(self) -> Parser:
        return get_parser(self.name)

python_language = Language("python")





